In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from torch.utils.data import DataLoader, TensorDataset
from xgboost import XGBRegressor
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
from load_data import process_steel_data

full_path = 'data/'
path = 'data/MDC_Data_Descriptions_MeCoMeP-r-value.xlsx'
correlation_rate = 0.2
dvl_line = 1

df = process_steel_data(full_path, path, correlation_rate, dvl_line, model_output=True)
df = pd.get_dummies(df, columns=['steel_family'], prefix='steel').drop(['steel_grade'], axis=1)

In [3]:
def scale_data(df, binary_prefix='steel_'):

    # Identify binary columns
    binary_columns = [col for col in df.columns if col.startswith(binary_prefix)]
    
    # Identify columns to scale (non-binary columns)
    columns_to_scale = [col for col in df.columns if col not in binary_columns + ['r_value']]
    
    # Scale numerical features
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[columns_to_scale])
    
    # Create new dataframe with scaled data
    scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)
    
    # Add back binary columns
    for col in binary_columns:
        scaled_df[col] = df[col].values
    
    # Add target variable if present
    if 'r_value' in df.columns:
        scaled_df['r_value'] = df['r_value'].values
    
    return scaled_df, scaler

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_scaled_df, scaler = scale_data(train_df)
binary_columns = [col for col in test_df.columns if col.startswith('steel_')]
columns_to_scale = [col for col in test_df.columns if col not in binary_columns + ['r_value']]
scaled_test_data = scaler.transform(test_df[columns_to_scale])
test_scaled_df = pd.DataFrame(scaled_test_data, columns=columns_to_scale)
for col in binary_columns:
    test_scaled_df[col] = test_df[col].values
if 'r_value' in test_df.columns:
    test_scaled_df['r_value'] = test_df['r_value'].values

In [14]:
bool_cols = train_scaled_df.select_dtypes(include='bool').columns

filtered_dfs_train = {col: train_scaled_df[train_scaled_df[col]].drop(columns=bool_cols) for col in bool_cols}
filtered_dfs_test = {col: test_scaled_df[test_scaled_df[col]].drop(columns=bool_cols) for col in bool_cols}

In [9]:
def train_model_with_cv_gridsearch(df, model, param_grid=None, n_splits=5, random_state=42, use_grid_search=True, model_params=None):
    """
    Train a model with optional grid search and cross-validation
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe
    model : estimator object
        Machine learning model to train
    param_grid : dict, optional
        Parameter grid for grid search (used if use_grid_search=True)
    n_splits : int, optional
        Number of cross-validation splits (default: 5)
    random_state : int, optional
        Random state for reproducibility (default: 42)
    use_grid_search : bool, optional
        Whether to perform grid search (default: True)
    model_params : dict, optional
        Direct model parameters to use if use_grid_search=False
    
    Returns:
    --------
    dict containing model results and performance metrics including tol90
    """
    # Prepare X and y
    X = df.drop(['r_value'], axis=1)
    y = df['r_value']
    
    # Initialize cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Initialize metrics storage
    cv_scores = {
        'mae': [],
        'mse': [],
        'r2': [],
        'tol90': []  # Add tol90 metric
    }
    
    # Determine model parameters
    if use_grid_search:
        if param_grid is None:
            raise ValueError("param_grid must be provided when use_grid_search is True")
        
        # Initialize GridSearchCV
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=n_splits,
            scoring='neg_mean_absolute_error',
            n_jobs=-1,
            verbose=0
        )
        
        # Fit GridSearchCV
        print("Performing GridSearch...")
        grid_search.fit(X, y)
        print(f"\nBest parameters: {grid_search.best_params_}")
        best_model = grid_search.best_estimator_
    else:
        # Use directly specified parameters or default model
        if model_params:
            best_model = type(model)(**model_params)
        else:
            best_model = model
        
        grid_search = None
    
    # Perform cross-validation
    print("\nPerforming cross-validation...")
    pbar = tqdm(enumerate(kf.split(X), 1),
                total=n_splits,
                desc="Cross-validation",
                leave=True)
    
    for fold, (train_idx, val_idx) in pbar:
        # Split data
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train model
        best_model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = best_model.predict(X_val)
        
        # Calculate metrics
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        # Calculate tol90 (90th percentile of absolute errors)
        abs_errors = np.abs(y_val - y_pred)
        tol90 = np.percentile(abs_errors, 90)
        
        cv_scores['mae'].append(mae)
        cv_scores['mse'].append(mse)
        cv_scores['r2'].append(r2)
        cv_scores['tol90'].append(tol90)
        
        # Update progress bar description
        pbar.set_description(
            f"Fold {fold} - MAE: {mae:.4f}, MSE: {mse:.4f}, R2: {r2:.4f}, TOL90: {tol90:.4f}"
        )
    
    # Prepare results
    results = {
        'model': best_model,
        'best_params': grid_search.best_params_ if use_grid_search else model_params or {},
        'avg_mae': np.mean(cv_scores['mae']),
        'std_mae': np.std(cv_scores['mae']),
        'avg_mse': np.mean(cv_scores['mse']),
        'std_mse': np.std(cv_scores['mse']),
        'avg_r2': np.mean(cv_scores['r2']),
        'std_r2': np.std(cv_scores['r2']),
        'avg_tol90': np.mean(cv_scores['tol90']),  # Add average tol90
        'std_tol90': np.std(cv_scores['tol90']),   # Add std of tol90
        'cv_scores': cv_scores,
        'grid_search_results': grid_search.cv_results_ if use_grid_search else None
    }
    
    return results

def report_cv_results(results_dict, test_dfs):
    """
    Report cross-validation results and evaluate model performance on test datasets.

    Parameters:
    -----------
    results_dict : dict
        Dictionary containing model training results for different steel families.
    test_dfs : dict
        Dictionary containing test datasets for each steel family.

    Returns:
    --------
    None
    """
    for steel_family, results in results_dict.items():
        print(f"\nCross-Validation and Test Results for {steel_family}:")
        print("-" * 50)
        print(f"Best Parameters: {results['best_params']}")
        print(f"Average MAE (CV): {results['avg_mae']:.4f} ± {results['std_mae']:.4f}")
        print(f"Average MSE (CV): {results['avg_mse']:.4f} ± {results['std_mse']:.4f}")
        print(f"Average R2 (CV): {results['avg_r2']:.4f} ± {results['std_r2']:.4f}")
        print(f"Average TOL90 (CV): {results['avg_tol90']:.4f} ± {results['std_tol90']:.4f}")

        # Ensure test data exists for this steel family
        if steel_family not in test_dfs:
            print(f"Warning: No test data found for {steel_family}. Skipping test evaluation.")
            continue

        test_df = test_dfs[steel_family]
        
        # Prepare X_test and y_test
        X_test = test_df.drop(['r_value'], axis=1)
        y_test = test_df['r_value']

        # Make predictions on test set
        y_pred_test = results['model'].predict(X_test)

        # Compute test set performance metrics
        test_mae = mean_absolute_error(y_test, y_pred_test)
        test_mse = mean_squared_error(y_test, y_pred_test)
        test_r2 = r2_score(y_test, y_pred_test)

        # Compute tol90 for test set
        abs_errors_test = np.abs(y_test - y_pred_test)
        test_tol90 = np.percentile(abs_errors_test, 90)

        print(f"Test MAE: {test_mae:.4f}")
        print(f"Test MSE: {test_mse:.4f}")
        print(f"Test R2: {test_r2:.4f}")
        print(f"Test TOL90: {test_tol90:.4f}")
        print("=" * 50)


In [ ]:
rfr = RandomForestRegressor(random_state=42)
rfr_param_grid = {
    'n_estimators': [350]
}

rfr_results_dict = {}

for steel_family in filtered_dfs_train.keys():
    print(f"Training model for steel family: {steel_family}")
    rfr_results_dict[steel_family] = train_model_with_cv_gridsearch(
        df=filtered_dfs_train[steel_family].select_dtypes(exclude='bool'),
        model=rfr,
        param_grid=rfr_param_grid,
        n_splits=5
    )
report_cv_results(rfr_results_dict, filtered_dfs_test)

In [ ]:
xgb_model = XGBRegressor(random_state=42)

xgb_param_grid = {
    'eta': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4],
    'lambda': [0, 0.01, 0.1, 1, 10, 50],
    'max_depth': [3, 4, 5, 6, 7, 8]
}

xgb_results_dict = {}

for steel_family in filtered_dfs_train.keys():
    print(f"Training model for steel family: {steel_family}")
    xgb_results_dict[steel_family] = train_model_with_cv_gridsearch(
        df=filtered_dfs_train[steel_family],
        model=xgb_model,
        param_grid=xgb_param_grid,
        n_splits=5
    )
report_cv_results(xgb_results_dict, filtered_dfs_test)

In [17]:
class MultiBranchSteelRegressor(nn.Module):
    def __init__(self, chemical_dim, time_dim, process_dim, model_dim, hidden_units=64, dropout_rate=0.2):
        super().__init__()
        # Track which branches are active
        self.has_chemical = chemical_dim > 0
        self.has_time = time_dim > 0
        self.has_process = process_dim > 0
        self.has_model = model_dim > 0
        
        # Count active branches
        self.active_branches = sum([self.has_chemical, self.has_time, self.has_process, self.has_model])
        
        # Adjust hidden units for each branch
        self.branch_hidden = min(hidden_units, max(16, hidden_units // 2))
        
        # Creating branch
        def create_branch(input_dim):
            return nn.Sequential(
                nn.Linear(input_dim, self.branch_hidden),
                nn.BatchNorm1d(self.branch_hidden),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            )
        
        # Only create branches that have features
        if self.has_chemical:
            self.chemical_branch = create_branch(chemical_dim)
        if self.has_time:
            self.time_branch = create_branch(time_dim)
        if self.has_process:
            self.process_branch = create_branch(process_dim)
        if self.has_model:
            self.model_branch = create_branch(model_dim)
        
        # Combined input dimension based on active branches only
        combined_dim = self.branch_hidden * self.active_branches
        
        # Final layers after concatenation
        self.final_layers = nn.Sequential(
            nn.Linear(combined_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1)
        )
    
    def forward(self, chemical, time, process, model):
        features = []
        # Only process branches that have features
        if self.has_chemical:
            if chemical.dim() == 1:
                chemical = chemical.unsqueeze(0)
            features.append(self.chemical_branch(chemical))
        
        if self.has_time:
            if time.dim() == 1:
                time = time.unsqueeze(0)
            features.append(self.time_branch(time))
        
        if self.has_process:
            if process.dim() == 1:
                process = process.unsqueeze(0)
            features.append(self.process_branch(process))
        
        if self.has_model:
            if model.dim() == 1:
                model = model.unsqueeze(0)
            features.append(self.model_branch(model))
        
        # Concatenate only active features
        combined = torch.cat(features, dim=1) if len(features) > 1 else features[0]
        return self.final_layers(combined)

In [21]:
# labeling the features for each branch
features = [col for col in df.columns if col not in ['r_value', 'steel_family', 'steel_grade']]
features_dict = {
   'time': [col for col in features if 'time' in col.lower()], 
   'chemical': ['pct_al', 'pct_b', 'pct_c', 'pct_cr', 'pct_mn', 'pct_n', 'pct_nb', 'pct_si', 'pct_ti', 'pct_v', 'mfia_coil_frac_fer', 'mfia_et1_frac_fer', 'mfia_et2_frac_fer'],
   'model': ["rm", "ag", "a80", "n_value"]
}
features_dict['process'] = [col for col in features if col not in features_dict['time'] and col not in features_dict['chemical']]

In [23]:
from sklearn.model_selection import ParameterGrid

def train_and_evaluate_models_gridsearch(filtered_dfs_train, filtered_dfs_test, features_dict, num_epochs, param_grid, use_l2=False):
    """
    Train and evaluate models for multiple steel families using filtered training and test datasets with Grid Search.

    Parameters:
    -----------
    filtered_dfs_train : dict
        Dictionary containing training datasets for each steel family.
    filtered_dfs_test : dict
        Dictionary containing test datasets for each steel family.
    features_dict : dict
        Dictionary defining feature categories (chemical, time, process, model).
    num_epochs : int
        Number of epochs for training.
    param_grid : dict
        Dictionary of hyperparameter values for grid search.
    use_l2 : bool, optional
        Whether to use L2 regularization (default: False).

    Returns:
    --------
    results_dict : dict
        Dictionary with best trained models and evaluation metrics for each steel family.
    """
    
    results_dict = {}
    grid = list(ParameterGrid(param_grid))  # Convert ParameterGrid to list of dicts

    for steel_family in filtered_dfs_train.keys():
        print(f"\nTraining model for steel family: {steel_family}")

        # Get train and test data
        train_df = filtered_dfs_train[steel_family]
        test_df = filtered_dfs_test.get(steel_family, None)

        # Initialize best metrics
        best_model = None
        best_metrics = None
        best_params = None
        best_r2 = -float("inf")  # Higher is better for R2 score

        for hyperparameters in grid:
            print(f"\nTrying hyperparameters: {hyperparameters}")

            batch_size = hyperparameters['batch_size']
            
            # Initialize feature arrays and dimensions
            feature_arrays = {}
            feature_dims = {}

            # Process each feature category
            for category in ['chemical', 'time', 'process', 'model']:
                available_features = [col for col in features_dict[category] if col in train_df.columns]

                if available_features:
                    feature_arrays[category] = train_df[available_features].values.astype(np.float32)
                    feature_dims[category] = len(available_features)
                else:
                    feature_arrays[category] = np.zeros((len(train_df), 0), dtype=np.float32)
                    feature_dims[category] = 0

            # Prepare targets
            targets = train_df['r_value'].values

            # Split data
            split_data = train_test_split(
                feature_arrays['chemical'],
                feature_arrays['time'],
                feature_arrays['process'],
                feature_arrays['model'],
                targets,
                test_size=0.2,
                random_state=42
            )

            (X_train_chem, X_test_chem, X_train_time, X_test_time, 
             X_train_proc, X_test_proc, X_train_model, X_test_model, 
             y_train, y_test) = split_data

            # Convert to tensors
            train_tensors = {
                'chemical': torch.FloatTensor(X_train_chem),
                'time': torch.FloatTensor(X_train_time),
                'process': torch.FloatTensor(X_train_proc),
                'model': torch.FloatTensor(X_train_model)
            }

            test_tensors = {
                'chemical': torch.FloatTensor(X_test_chem),
                'time': torch.FloatTensor(X_test_time),
                'process': torch.FloatTensor(X_test_proc),
                'model': torch.FloatTensor(X_test_model)
            }

            y_train_tensor = torch.FloatTensor(y_train)
            y_test_tensor = torch.FloatTensor(y_test)

            # Create DataLoader
            train_dataset = TensorDataset(
                train_tensors['chemical'],
                train_tensors['time'],
                train_tensors['process'],
                train_tensors['model'],
                y_train_tensor
            )
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

            # Initialize model
            model = MultiBranchSteelRegressor(
                chemical_dim=feature_dims['chemical'],
                time_dim=feature_dims['time'],
                process_dim=feature_dims['process'],
                model_dim=feature_dims['model'],
                hidden_units=hyperparameters['hidden_units'],
                dropout_rate=hyperparameters['dropout_rate']
            )

            if use_l2:
                weight_decay = 0.001
            else:
                weight_decay = 0.0

            optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters['learning_rate'], weight_decay=weight_decay)
            criterion = nn.L1Loss()

            # Training loop
            model.train()
            for epoch in range(num_epochs):
                running_loss = 0.0
                for batch_chem, batch_time, batch_proc, batch_model, batch_targets in train_loader:
                    optimizer.zero_grad()
                    outputs = model(batch_chem, batch_time, batch_proc, batch_model)
                    loss = criterion(outputs, batch_targets.unsqueeze(1))
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()

                if (epoch + 1) % 10 == 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

            # Evaluation
            model.eval()
            with torch.no_grad():
                y_pred = model(
                    test_tensors['chemical'],
                    test_tensors['time'],
                    test_tensors['process'],
                    test_tensors['model']
                )
                test_loss = criterion(y_pred, y_test_tensor.unsqueeze(1)).item()
                y_pred_np = y_pred.numpy().flatten()
                r2 = r2_score(y_test, y_pred_np)
                mae = mean_absolute_error(y_test, y_pred_np)
                mse = mean_squared_error(y_test, y_pred_np)

                abs_errors = np.abs(y_test - y_pred_np)
                tol90 = np.percentile(abs_errors, 90)

                metrics = {
                    'model': model,
                    'best_params': hyperparameters,
                    'avg_mae': mae,
                    'std_mae': 0.0,
                    'avg_mse': mse,
                    'std_mse': 0.0,
                    'avg_r2': r2,
                    'std_r2': 0.0,
                    'avg_tol90': tol90,
                    'std_tol90': 0.0,
                    'grid_search_results': None
                }

                print(f"\nEvaluation - {steel_family}: R2: {r2:.4f}, MAE: {mae:.4f}, Test Loss: {test_loss:.4f}")

                # Save best model based on R2 score
                if r2 > best_r2:
                    best_r2 = r2
                    best_model = model
                    best_metrics = metrics
                    best_params = hyperparameters

        # Store best results for this steel family
        if best_model:
            results_dict[steel_family] = best_metrics

    return results_dict

In [ ]:
param_grid = {
    'learning_rate': [0.01, 1e-3],
    'batch_size': [16, 32, 64],
    'hidden_units': [64, 128, 256],
    'dropout_rate': [0, 0.2]
}
num_epochs = 100

results_dict = train_and_evaluate_models_gridsearch(filtered_dfs_train, filtered_dfs_test, features_dict, num_epochs, param_grid)

report_cv_results(results_dict, filtered_dfs_test)